# tsam - Method Showcase

Examples of different clustering methods in the time series aggregation module (tsam).

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

import tsam
from tsam import ClusterConfig

%matplotlib inline

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

### Simple k-means aggregation

Use k-means clustering for eight typical days. Other methods include: `"hierarchical"`, `"kmedoids"`, `"averaging"`.

In [ ]:
result_kmeans = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24,
    cluster=ClusterConfig(method="kmeans"),
)

Create the typical periods

In [ ]:
typPeriods = result_kmeans.typical_periods
typPeriods.head()

Save typical periods to .csv file

In [ ]:
typPeriods.to_csv(os.path.join("results", "testperiods_kmeans.csv"))

### Simple k-medoids aggregation of weeks

Use k-medoids clustering for weekly periods. K-medoids selects actual periods as cluster centers (instead of computed centroids).

In [ ]:
result_kmedoids = tsam.aggregate(
    raw,
    n_periods=8,
    period_hours=24 * 7,  # Weekly periods
    cluster=ClusterConfig(method="kmedoids"),
)

Create the typical periods

In [ ]:
typPeriods = result_kmedoids.typical_periods
typPeriods.head()

Save typical periods to .csv file with weeks order by GHI to get later testing consistency

In [ ]:
typPeriods.reindex(
    typPeriods["GHI"].unstack().sum(axis=1).sort_values().index, level=0
).to_csv(os.path.join("results", "testperiods_kmedoids.csv"))

The aggregation can also be evaluated by indicators

In [ ]:
# View accuracy metrics
print(result_kmedoids.accuracy)